In [1]:
from get_data import get_videos_from_folder,get_target_from_csv
import os
import numpy as np
from utils import save_solution

dir_path = ""
train_folder = os.path.join(dir_path,"../train/")
test_folder = os.path.join(dir_path,"../test/")

train_target = os.path.join(dir_path,'../train_target.csv')
my_solution_file = os.path.join(dir_path,'../solution.csv')

x_train = get_videos_from_folder(train_folder)
y_train = get_target_from_csv(train_target)
x_test = get_videos_from_folder(test_folder)


In [16]:
max([len(x) for x in x_train])

209

In [31]:
X = []
for idx, v in enumerate(x_train):
    s = np.array([])
    for frame in v:
        s = np.hstack((s, frame.reshape(10000)))
    print("video %d: %d frames" % (idx, len(s) / 10000))
    s = np.pad(s, (0,2090000 - len(s)), 'constant')
    X.append(s)

video 0: 77 frames
video 1: 32 frames
video 2: 53 frames
video 3: 48 frames
video 4: 50 frames
video 5: 48 frames
video 6: 101 frames
video 7: 87 frames
video 8: 80 frames
video 9: 91 frames
video 10: 50 frames
video 11: 66 frames
video 12: 84 frames
video 13: 59 frames
video 14: 52 frames
video 15: 77 frames
video 16: 62 frames
video 17: 22 frames
video 18: 77 frames
video 19: 39 frames
video 20: 71 frames
video 21: 118 frames
video 22: 72 frames
video 23: 70 frames
video 24: 72 frames
video 25: 72 frames
video 26: 83 frames
video 27: 93 frames
video 28: 82 frames
video 29: 32 frames
video 30: 74 frames
video 31: 76 frames
video 32: 43 frames
video 33: 45 frames
video 34: 97 frames
video 35: 106 frames
video 36: 49 frames
video 37: 54 frames
video 38: 56 frames
video 39: 45 frames
video 40: 36 frames
video 41: 72 frames
video 42: 34 frames
video 43: 41 frames
video 44: 38 frames
video 45: 77 frames
video 46: 67 frames
video 47: 79 frames
video 48: 56 frames
video 49: 96 frames
video 5

In [35]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.model_selection import train_test_split

In [36]:
train_X, eval_X, train_Y, eval_Y = train_test_split(X, y_train, test_size=0.2, random_state=42)
xg_train = xgb.DMatrix(train_X, label=train_Y)
xg_eval = xgb.DMatrix(eval_X, label=eval_Y)

# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['booster'] = 'gbtree'
param['objective'] = 'binary:logistic'
# scale weight of positive examples
param['eta'] = 0.2
param['gamma'] = 1.0
param['max_depth'] = 6
param['silent'] = 1
param['subsample'] = 0.8
param['colsample_bytree'] = 0.9
param['min_child_weight'] = 20
param['num_class'] = 4


watchlist = [(xg_train, 'train'), (xg_eval, 'eval')]
num_round = 60
bst = xgb.train(param, 
                xg_train, 
                num_round, 
                watchlist, 
                feval=lambda y,t: ("ROC_AUC", roc_auc_score(y, t.get_label())))

# get prediction
pred = bst.predict(xg_eval)
# error_rate = np.sum(pred != test_Y) / test_Y.shape[0]
err = roc_auc_score(eval_Y, pred)
print('Test error using softmax = {}'.format(F1))

XGBoostError: b'[02:12:49] src/objective/regression_obj.cu:66: Check failed: preds.Size() == info.labels_.Size() (504 vs. 126) labels are not correctly providedpreds.size=504, label.size=126\n\nStack trace returned 7 entries:\n[bt] (0) 0   libxgboost.dylib                    0x0000001c276a3826 dmlc::StackTrace() + 246\n[bt] (1) 1   libxgboost.dylib                    0x0000001c276a35df dmlc::LogMessageFatal::~LogMessageFatal() + 47\n[bt] (2) 2   libxgboost.dylib                    0x0000001c277273ed xgboost::obj::RegLossObj<xgboost::obj::LogisticClassification>::GetGradient(xgboost::HostDeviceVector<float> const&, xgboost::MetaInfo const&, int, xgboost::HostDeviceVector<xgboost::detail::GradientPairInternal<float> >*) + 589\n[bt] (3) 3   libxgboost.dylib                    0x0000001c2769fd34 xgboost::LearnerImpl::UpdateOneIter(int, xgboost::DMatrix*) + 996\n[bt] (4) 4   libxgboost.dylib                    0x0000001c276ba6b4 XGBoosterUpdateOneIter + 180\n[bt] (5) 5   libffi.6.dylib     

In [ ]:
xg_test = xgb.DMatrix(test_X)
y_pred = bst.predict(xg_test)
f = open("submission.csv", "w")
f.write("id,y\n")
for i,x in enumerate(y_pred):
    f.write("{},{}\n".format(i,y_pred[i]))
f.close()